In [1]:
import heapq

class Node:
    """
    A node class for A* Pathfinding.
    Each node has a position, a parent, and g, h, f scores.
    """
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position  # (row, col)

        self.g = 0  # Cost from start to current node
        self.h = 0  # Heuristic cost from current node to end
        self.f = 0  # Total cost (g + h)

    def __eq__(self, other):
        return self.position == other.position

    # A* uses a priority queue, so we need to define ordering
    def __lt__(self, other):
        return self.f < other.f

    def __hash__(self):
        return hash(self.position)

def a_star_search(maze, start_pos, end_pos, heuristic_func):
    """
    Returns a list of tuples as a path from the given start to the given end in the given maze.
    Uses the provided heuristic function.
    """
    # Create start and end node
    start_node = Node(None, start_pos)
    end_node = Node(None, end_pos)

    # Initialize both open and closed list
    open_list = []
    closed_list = set()

    # Heapify the open_list and add the start node
    heapq.heappush(open_list, start_node)

    # Loop until you find the end
    while len(open_list) > 0:
        # Get the current node (the one with the smallest f value)
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or \
               node_position[1] > (len(maze[len(maze)-1]) - 1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] == 1:
                continue

            # Create new node
            new_node = Node(current_node, node_position)
            children.append(new_node)

        # Loop through children
        for child in children:
            # Child is on the closed list
            if child in closed_list:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = heuristic_func(child, end_node)
            child.f = child.g + child.h

            # Child is already in the open list
            # Check if this path is better (lower g score)
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            heapq.heappush(open_list, child)
            
    return None # Path not found

def print_maze(maze, path=[]):
    """Prints the maze with the path marked."""
    path_points = set(path)
    for r, row in enumerate(maze):
        for c, cell in enumerate(row):
            if (r, c) in path_points:
                if cell == 'A':
                    print('A ', end="")
                elif cell == 'B':
                    print('B ', end="")
                else:
                    print('* ', end="")
            elif cell == 1:
                print('# ', end="")
            elif cell == 'A' or cell == 'B':
                 print(f"{cell} ", end="")
            else:
                print('. ', end="")
        print() # Newline for next row



In [3]:
# --- Heuristic Functions ---

def manhattan_distance(node, end_node):
    """Standard admissible and consistent Manhattan distance heuristic."""
    return abs(node.position[0] - end_node.position[0]) + abs(node.position[1] - end_node.position[1])

def non_admissible_heuristic(node, end_node):
    """
    A non-admissible heuristic.
    Multiplying by 1.5 can cause it to overestimate the cost, since the cost of movement is 1.
    This can lead to a suboptimal path.
    """
    return manhattan_distance(node, end_node) * 1.5

def inconsistent_heuristic(node, end_node):
    """
    An admissible but inconsistent heuristic.
    It gives an artificially low value near a specific point (3,2) to violate the triangle inequality.
    For the transition from (3,1) to (3,2):
    h(3,1) = Manhattan = |3-2|+|1-2| = 2
    h(3,2) = Special case = 0
    cost( (3,1)->(3,2) ) = 1
    Check: h(3,1) <= cost + h(3,2)  =>  2 <= 1 + 0 => 2 <= 1 (FALSE) -> Inconsistent.
    """
    if node.position == (3, 2):
        return 0 # Artificially low value
    return manhattan_distance(node, end_node)




In [5]:
def main():
    # Maze setup: A=Start, B=Goal, 1=Wall, 0=Path
    maze = [
        ['A', 0,   0,   0,   0],
        [0,   1,   1,   1,   0],
        [0,   1,   'B', 1,   0],
        [0,   1,   0,   0,   0],
        [0,   0,   0,   1,   0]
    ]
    
    start = (0, 0)
    end = (2, 2)
    
    # --- Experiment 1: Standard A* with Manhattan Distance ---
    print("Experiment 1: Manhattan Distance (Admissible & Consistent)")
    path = a_star_search(maze, start, end, manhattan_distance)
    if path:
        path_cost = len(path) - 1
        print(f"Path found. Cost: {path_cost}")
        print("Is it optimal? Yes.")
        print_maze(maze, path)
    else:
        print("No path found.")
    print("-" * 50)
    
    # --- Experiment 2: Non-Admissible Heuristic ---
    print("\nExperiment 2: Non-Admissible Heuristic (Manhattan * 1.5)")
    # In this specific maze, the heuristic may still find the optimal path
    # because the suboptimal path is significantly longer.
    # The key is that non-admissibility *removes the guarantee* of optimality.
    path_non_admissible = a_star_search(maze, start, end, non_admissible_heuristic)
    if path_non_admissible:
        path_cost = len(path_non_admissible) - 1
        print(f"Path found. Cost: {path_cost}")
        print("Is it optimal? Not Guaranteed (but is optimal for this specific maze).")
        print("A non-admissible heuristic may overestimate costs, potentially leading to suboptimal paths in other mazes.")
        print_maze(maze, path_non_admissible)
    else:
        print("No path found.")
    print("-" * 50)

    # --- Experiment 3: Inconsistent Heuristic ---
    print("\n Experiment 3: Inconsistent (but Admissible) Heuristic")
    path_inconsistent = a_star_search(maze, start, end, inconsistent_heuristic)
    if path_inconsistent:
        path_cost = len(path_inconsistent) - 1
        print(f"Path found. Cost: {path_cost}")
        print("Is it optimal? Yes.")
        print("While inconsistent, this heuristic is still admissible, so the path is optimal.")
        print("However, the search may be inefficient, re-opening already visited nodes.")
        print_maze(maze, path_inconsistent)
    else:
        print("No path found.")
    print("-" * 50)


if __name__ == '__main__':
    main()


Experiment 1: Manhattan Distance (Admissible & Consistent)
Path found. Cost: 8
Is it optimal? Yes.
A . . . . 
* # # # . 
* # B # . 
* # * . . 
* * * # . 
--------------------------------------------------

Experiment 2: Non-Admissible Heuristic (Manhattan * 1.5)
Path found. Cost: 8
Is it optimal? Not Guaranteed (but is optimal for this specific maze).
A non-admissible heuristic may overestimate costs, potentially leading to suboptimal paths in other mazes.
A . . . . 
* # # # . 
* # B # . 
* # * . . 
* * * # . 
--------------------------------------------------

 Experiment 3: Inconsistent (but Admissible) Heuristic
Path found. Cost: 8
Is it optimal? Yes.
While inconsistent, this heuristic is still admissible, so the path is optimal.
However, the search may be inefficient, re-opening already visited nodes.
A . . . . 
* # # # . 
* # B # . 
* # * . . 
* * * # . 
--------------------------------------------------
